# Binary case only in One classifier each name: OCEN 

1. This method throw away the authors write less than 100 papers  

2. We only deal with binary case

Also this script evaluate different text embedding method with OCEN.

In [13]:
import os
import sys
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import com_func

# parameters
#----- filter for selecting set of name group -----------#
filter_select_name_group = 100
#----- filter for selecting productive authors ----#
filter_lower = 100
filter_upper = 110

Dataset = "pubmed"

In [10]:
# text embedding only
pp_text_emb = ["tf", "tf_idf", "lsa", "pv_dm", "pv_dbow"]
pp_citation_emb = ["off"]

In [14]:
# citation embedding only
pp_text_emb = ["off"]
pp_citation_emb = ["n2v"]
pp_citation_emb = ["node2vec"]

In [18]:
# combined embedding
pp_text_emb = ["lsa", "pv_dm", "pv_dbow"]
pp_citation_emb = ["n2v"]

In [19]:
print(pp_text_emb)
print(pp_citation_emb)

['lsa', 'pv_dm', 'pv_dbow']
['n2v']


In [16]:
import copy
import random
from collections import Counter
from sklearn.model_selection import StratifiedKFold

# cross validation
def k_fold_cv_with_different_train_size(data, label, clf, train_size=1, k=10):
    '''
    Split train and test for each fold first, then reduce train size
    train_size: between 0-1 is percentage, larger than one is train size count
    '''
    random.seed(1)
    if train_size<=0:
        sys.exit("Training size must be larger than 0")
    # sync input datatype
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)
    if not isinstance(label, pd.Series):
        label = pd.Series(label, index = data.index.values)
        
    # obtain data ratio
    c = collections.Counter(label)
    data_ratio = [(i, c[i] / len(label)) for i in c]
    print(data_ratio)
    
    kf = StratifiedKFold(n_splits=k)
    allTrueLabel = []
    allPredLabel = []
    all_fold_statistic = []
    test_size = 0
    fold = 0
    
    for train_index, test_index in kf.split(data, label):
        fold +=1
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # ---------------1. split train and test -------------------- #
        data_train, data_test = data.iloc[train_index], data.iloc[test_index]
        all_label_train, label_test = label.iloc[train_index], label.iloc[test_index]
        # ---------------2. train classifier with different training size -------------------- #
        if train_size<=1:
            train_size = len(all_label_train)*train_size
        if train_size > len(all_label_train):
            sys.exit("Training size must be less or equal to total training samples")
        # if train_size float, take floor of a float train_size
        train_size = int(train_size)
        test_size = len(data_test)
        # 1. number of samples for each class when perserve it's data ratio
        train_per_class_size = [(label, round(ratio*train_size)) for label, ratio in data_ratio]
        selected_train_sample_idx = []
        print(train_per_class_size)
        # 2. select samples from train using variable we generated 
        for unique_label, training_size in train_per_class_size:
            curr_label_idx = all_label_train.index[all_label_train == unique_label].tolist()
            curr_label_size = len(curr_label_idx)
            selected_train_sample_idx += random.sample(curr_label_idx, training_size)
        # .loc use index, .iloc use position
        final_data_train = data_train.loc[selected_train_sample_idx]
        final_label_train = all_label_train.loc[selected_train_sample_idx]
        # 3. train classifier
        per_fold_clf = copy.deepcopy(clf)
        per_fold_clf.fit(final_data_train, final_label_train)
        # 4. make predcit on test
        per_fold_predict_test = per_fold_clf.predict(data_test)
                
        allTrueLabel.extend(label_test.values.tolist())
        allPredLabel.extend(per_fold_predict_test)
        # collect per fold statistic
        curr_fold_statistic = {'fold':fold, 'train_size': train_per_class_size, 'test_size': data_test.shape[0],
                               'macro f1': f1_score(label_test, per_fold_predict_test,average='macro')}
        all_fold_statistic.append(curr_fold_statistic)
        
    # macro weighs each class equally 
    # micro weights each sample equally.
    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    macro_f1 = f1_score(allTrueLabel, allPredLabel,average='macro')
    
    return accuracy, macro_f1, train_size, test_size

In [20]:
# load the file
import io
import collections
import numpy as np
import pandas as pd

from statistics import mean 

from sklearn.preprocessing import normalize
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# fix random seed for reproducibility
np.random.seed(1)

fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)

# ----------------------- different text embedding ----------------------#
for text_emb in pp_text_emb:
    print("Load text embedding: ", text_emb)
    # read pretrained embeddings
    all_text_embedding, all_text_emb_pid = com_func.read_text_embedding(emb_type=text_emb, training_size="140k")
    
    for citation_emb in pp_citation_emb:
        print("Load citation embedding: ", citation_emb)
        all_citation_embedding = com_func.read_citation_embedding_sorted(emb_type = citation_emb)
        all_citation_emb_pid = [emb[0] for emb in all_citation_embedding]
        all_citation_embedding = [emb[1:] for emb in all_citation_embedding]

        # -------------- different filter (step by 10) -----------------------#
        for step_filter in range(filter_lower, filter_upper, 10):
            # collect statistic to output
            allname, num_class, per_class_count, selected_sample_size, orginal_sample_size= ([] for i in range(5))
            
            train_percent_change = []

            all_mnb_accuracy, all_mnb_f1, all_LR_accuracy = ([] for i in range(3))
            all_LR_f1, all_svcLinear_accuracy, all_svcLinear_f1 = ([] for i in range(3))

            total_selected_group = 0

            # ------- select useful name group in all name group --------------------#
            for file in listfiles:
                # group name
                temp = file.split("_")
                name = temp[1]+"_"+temp[-1]
                print("For name: ",name)
                # read needed content in labeled file
                labeled_data = com_func.read_pid_aid(fileDir+file)
                #----------- select name group contain productive author------------------------------------#
                #----------- (contain pair of author write more than 100 papers) ---------------------------#
                # count number of paper each author write based on author ID
                authorCounter = collections.Counter(labeled_data["authorID"])
                # remove name group that do not contain pair of author write more than 100 papers
                for k in list(authorCounter):
                    if authorCounter[k] < filter_select_name_group:
                        del authorCounter[k]
                # if only have one class or no class pass the filter, not applicable
                if(len(authorCounter)==0) or (len(authorCounter)==1):
                    print(name, " pass")
                elif (len(authorCounter)!=2):
                    print(name+ " is multi-class case, ignored")
                else:
                    print(name + " is binary case")
                    total_selected_group+= 1
                    temp_orginal_sample_size = len(labeled_data)
                    #--------select authors in name group are very productive (more than filter)---------#
                    print("Total sample size before apply filter: ",len(labeled_data))
                    # count number of paper each author write based on author ID
                    paperCounter = collections.Counter(labeled_data["authorID"])
                    print(paperCounter)
                    print("Total author before apply threshoid: ", len(paperCounter))
                    # collect per class statistic
                    for k in list(paperCounter):
                        if paperCounter[k] < step_filter:
                            del paperCounter[k]
                    temp =list(paperCounter.keys())
                    print(temp)
                    print("Total author after apply threshoid: ", len(temp))
                    # remove samples that are smaller than filter
                    labeled_data = labeled_data[labeled_data.authorID.isin(temp)]
                    print("Total sample size after apply filter: ",len(labeled_data))
                    #------------ extract paper representation -------------------------------------------#
                    # shuffle the data
                    labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
                    # extract true label and pid
                    label = labeled_data["authorID"]
                    pid = labeled_data["paperID"]
                    # list of different data field
                    part_collection = []
                    # select feature wanted to fit to clustering/classification algorithm
                    # data part, text information
                    data_part_text = com_func.extract_embedding(all_text_embedding, all_text_emb_pid, pid)
                    print("Text embedding shape: ", data_part_text.shape)
                    part_collection.append(data_part_text)
                    # data part, citation information
                    data_part_citation = com_func.extract_embedding(all_citation_embedding, all_citation_emb_pid, pid)
                    print("Citation embedding shape: ", data_part_citation.shape)
                    part_collection.append(data_part_citation)
                    # merge different part of data data together by concatenate it all together
                    # remove empty emb (when emb set off)
                    part_collection = [part for part in part_collection if len(part)!=0]
                    if len(part_collection)>1:
                        combinedata = np.concatenate(part_collection,axis=1)
                    elif len(part_collection)==1:
                        if isinstance(part_collection[0], pd.DataFrame):
                            combinedata = part_collection[0].values
                        else:
                            combinedata = part_collection[0]
                    else:
                        print("No data available")
                        break
                    print("Final feature (combined embedding) shape: ", combinedata.shape)
                    # ------------- 10% to 100% training size changes -----------------------#
                    for train_percent in np.arange(0.1, 1.1, 0.1):
                        allname.append(name)
                        num_class.append(len(paperCounter))
                        per_class_count.append(paperCounter)
                        selected_sample_size.append(len(labeled_data))
                        orginal_sample_size.append(temp_orginal_sample_size)
                        train_percent_change.append(train_percent)
                        # -------------- using converted feature vector to train classifier-------------------#
                        if text_emb == "tf":
                            # using multinomial naive bayes
                            clf = MultinomialNB()
                            mnbaccuracy, mnbmarcof1, train_size, test_size = k_fold_cv_with_different_train_size(combinedata, label, clf, train_size=train_percent, k=10)
                            print("MNB F1: ", mnbmarcof1)
                            all_mnb_accuracy.append(mnbaccuracy)
                            all_mnb_f1.append(mnbmarcof1)
                        # using logistic regression
                        clf = LogisticRegression(solver= "liblinear")
                        LRaccuracy, LRmarcof1, train_size, test_size = k_fold_cv_with_different_train_size(combinedata, label, clf, train_size=train_percent, k=10)
                        print("LR F1: ", LRmarcof1)
                        all_LR_accuracy.append(LRaccuracy)
                        all_LR_f1.append(LRmarcof1)
                        # using SVM with linear kernal
                        clf = SVC(gamma="auto", kernel='linear')
                        svcaccuracy, svcmarcof1, train_size, test_size = k_fold_cv_with_different_train_size(combinedata, label, clf, train_size=train_percent, k=10)
                        print("svc F1: ", svcmarcof1)
                        all_svcLinear_accuracy.append(svcaccuracy)
                        all_svcLinear_f1.append(svcmarcof1)

            if text_emb == "tf":
                # write evaluation result to excel
                output = pd.DataFrame({'Name Group':allname,"Class number":num_class,"Per class size":per_class_count, 
                                       "Total selected sample size":selected_sample_size, "Orginal sample size":orginal_sample_size, 
                                       "used_train_percent":train_percent_change,
                                       "MNB Accuracy":all_mnb_accuracy, "MNB macro F1": all_mnb_f1, 
                                       "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) macro f1": all_svcLinear_f1, 
                                       "logistic regression accuracy":all_LR_accuracy, "logistic regression macro f1": all_LR_f1})
            else:
                # write evaluation result to excel
                output = pd.DataFrame({'Name Group':allname, "Class number":num_class, "Per class size":per_class_count,
                                       "Total selected sample size":selected_sample_size,"Orginal sample size":orginal_sample_size,
                                       "used_train_percent":train_percent_change,
                                       "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) macro f1": all_svcLinear_f1, 
                                       "logistic regression accuracy":all_LR_accuracy, "logistic regression macro f1": all_LR_f1})

            savePath = "../../result/"+Dataset+"/binary_case_only_textEmb_sample=140k/"
            filename = "(Global emb sample 140k) citationEmb="+citation_emb+"_textEmb="+text_emb+"_filter="+str(step_filter)+"_namegroupcount="+str(total_selected_group)+".csv"
            com_func.write_csv_df(savePath, filename, output)
            print("Done")

Load text embedding:  lsa
Total text vector records: 135796
Vector dimension:  100
Load citation embedding:  n2v
Total citation vector records: 124922
Vector dimension:  101
For name:  j_read
j_read  pass
For name:  f_esteves
f_esteves  pass
For name:  c_miller
c_miller  pass
For name:  r_jha
r_jha  pass
For name:  a_lowe
a_lowe  pass
For name:  a_vega
a_vega  pass
For name:  k_smith
k_smith  pass
For name:  j_gordon
j_gordon  pass
For name:  s_liao
s_liao  pass
For name:  j_qian
j_qian  pass
For name:  s_bernardi
s_bernardi  pass
For name:  t_hill
t_hill  pass
For name:  s_schindler
s_schindler  pass
For name:  j_williams
j_williams  pass
For name:  s_jacobson
s_jacobson  pass
For name:  e_andrade
e_andrade  pass
For name:  t_santos
t_santos  pass
For name:  k_kim
k_kim is multi-class case, ignored
For name:  d_ricci
d_ricci  pass
For name:  s_cameron
s_cameron  pass
For name:  t_wright
t_wright  pass
For name:  r_cunha
r_cunha  pass
For name:  s_fuchs
s_fuchs  pass
For name:  m_nawaz

[('0000-0002-0736-9199', 21), ('0000-0002-7878-0313', 24)]
[('0000-0002-0736-9199', 21), ('0000-0002-7878-0313', 24)]
[('0000-0002-0736-9199', 21), ('0000-0002-7878-0313', 24)]
[('0000-0002-0736-9199', 21), ('0000-0002-7878-0313', 24)]
[('0000-0002-0736-9199', 21), ('0000-0002-7878-0313', 24)]
[('0000-0002-0736-9199', 21), ('0000-0002-7878-0313', 24)]
svc F1:  0.9361823361823362
[('0000-0002-0736-9199', 0.4722222222222222), ('0000-0002-7878-0313', 0.5277777777777778)]
[('0000-0002-0736-9199', 32), ('0000-0002-7878-0313', 35)]
[('0000-0002-0736-9199', 32), ('0000-0002-7878-0313', 35)]
[('0000-0002-0736-9199', 32), ('0000-0002-7878-0313', 35)]
[('0000-0002-0736-9199', 32), ('0000-0002-7878-0313', 35)]
[('0000-0002-0736-9199', 32), ('0000-0002-7878-0313', 35)]
[('0000-0002-0736-9199', 32), ('0000-0002-7878-0313', 35)]
[('0000-0002-0736-9199', 32), ('0000-0002-7878-0313', 35)]
[('0000-0002-0736-9199', 32), ('0000-0002-7878-0313', 35)]
[('0000-0002-0736-9199', 32), ('0000-0002-7878-0313', 3

[('0000-0002-0736-9199', 85), ('0000-0002-7878-0313', 95)]
[('0000-0002-0736-9199', 85), ('0000-0002-7878-0313', 95)]
[('0000-0002-0736-9199', 85), ('0000-0002-7878-0313', 95)]
[('0000-0002-0736-9199', 85), ('0000-0002-7878-0313', 95)]
[('0000-0002-0736-9199', 85), ('0000-0002-7878-0313', 95)]
[('0000-0002-0736-9199', 85), ('0000-0002-7878-0313', 95)]
[('0000-0002-0736-9199', 85), ('0000-0002-7878-0313', 95)]
[('0000-0002-0736-9199', 85), ('0000-0002-7878-0313', 95)]
[('0000-0002-0736-9199', 85), ('0000-0002-7878-0313', 95)]
svc F1:  0.9441595441595442
[('0000-0002-0736-9199', 0.4722222222222222), ('0000-0002-7878-0313', 0.5277777777777778)]
[('0000-0002-0736-9199', 96), ('0000-0002-7878-0313', 107)]
[('0000-0002-0736-9199', 96), ('0000-0002-7878-0313', 107)]
[('0000-0002-0736-9199', 96), ('0000-0002-7878-0313', 107)]
[('0000-0002-0736-9199', 96), ('0000-0002-7878-0313', 107)]
[('0000-0002-0736-9199', 96), ('0000-0002-7878-0313', 107)]
[('0000-0002-0736-9199', 96), ('0000-0002-7878-031

[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
LR F1:  0.9923460612315101
[('0000-0002-3650-9381', 0.5767790262172284), ('0000-0003-1673-2954', 0.4232209737827715)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
[('0000-0002-3650-9381', 41), ('0000-0003-1673-2954', 30)]
svc F1:  0.9847254004576659
[('0000-0002-3650-9381', 0.5767790262172284), ('0000-0003-1673-2954', 0.4232209737827715)]
[('0000-0002-3650-9381', 55), ('0000-0003-1673-2954', 40)]
[('0000-0002-3650-9381', 55), ('0000-0003-1673-2954', 4

[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
LR F1:  0.9923460612315101
[('0000-0002-3650-9381', 0.5767790262172284), ('0000-0003-1673-2954', 0.4232209737827715)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
[('0000-0002-3650-9381', 124), ('0000-0003-1673-2954', 91)]
svc F1:  0.9885317488725034
[('0000-0002-3650-9381', 0.5767790262172284), ('0000-0003-1673-2954', 0.42

...

Missing Sample:  25596741
Missing Sample:  27331625
Missing Sample:  20617322
Missing Sample:  26603572
Missing Sample:  27573925
Missing Sample:  23232914
Missing Sample:  19994868
Missing Sample:  18651769
Missing Sample:  26862851
Missing Sample:  26694251
Missing Sample:  22791448
Missing Sample:  15360326
Missing Sample:  24301072
Missing Sample:  25992645
Missing Sample:  24163332
Missing Sample:  19182891
Missing Sample:  26504932
Total missing sample:  36
Citation embedding shape:  (536, 100)
Final feature (combined embedding) shape:  (536, 200)
[('0000-0002-4295-6129', 0.789179104477612), ('0000-0002-9639-7940', 0.21082089552238806)]
[('0000-0002-4295-6129', 38), ('0000-0002-9639-7940', 10)]
[('0000-0002-4295-6129', 38), ('0000-0002-9639-7940', 10)]
[('0000-0002-4295-6129', 38), ('0000-0002-9639-7940', 10)]
[('0000-0002-4295-6129', 38), ('0000-0002-9639-7940', 10)]
[('0000-0002-4295-6129', 38), ('0000-0002-9639-7940', 10)]
[('0000-0002-4295-6129', 38), ('0000-0002-9639-7940', 

[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
LR F1:  0.9915067581486
[('0000-0002-4295-6129', 0.789179104477612), ('0000-0002-9639-7940', 0.21082089552238806)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
[('0000-0002-4295-6129', 227), ('0000-0002-9639-7940', 61)]
svc F1:  0.969062049062049
[('0000-0002-4295-6129', 0.789179104477612), ('0000-0002-9639-7940', 0.2108208

j_petersen  pass
For name:  k_shimizu
k_shimizu  pass
For name:  p_shaw
p_shaw  pass
For name:  g_coppola
g_coppola  pass
For name:  a_sinclair
a_sinclair  pass
For name:  y_pan
y_pan  pass
For name:  m_ramos
m_ramos  pass
For name:  j_tsai
j_tsai  pass
For name:  f_dai
f_dai  pass
For name:  t_martin
t_martin  pass
For name:  t_o'brien
t_o'brien  pass
For name:  s_may
s_may  pass
For name:  z_cai
z_cai  pass
For name:  a_pereira
a_pereira  pass
For name:  d_patel
d_patel  pass
For name:  a_james
a_james  pass
For name:  c_cao
c_cao  pass
For name:  c_brown
c_brown  pass
For name:  y_liang
y_liang  pass
For name:  y_fan
y_fan  pass
For name:  j_simon
j_simon  pass
For name:  m_jeong
m_jeong  pass
For name:  j_barrett
j_barrett  pass
For name:  d_elliott
d_elliott  pass
For name:  p_antunes
p_antunes  pass
For name:  x_yuan
x_yuan  pass
For name:  t_kim
t_kim  pass
For name:  a_cruz
a_cruz  pass
For name:  a_mora
a_mora  pass
For name:  j_walker
j_walker  pass
For name:  j_alves
j_alves

[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
[('0000-0002-9152-6512', 95), ('0000-0003-3152-116X', 65)]
svc F1:  0.9836884520524273
[('0000-0002-9152-6512', 0.5955056179775281), ('0000-0003-3152-116X', 0.4044943820224719)]
[('0000-0002-9152-6512', 119), ('0000-0003-3152-116X', 81)]
[('0000-0002-9152-6512', 119), ('0000-0003-3152-116X', 81)]
[('0000-0002-9152-6512', 119), ('0000-0003-3152-116X', 81)]
[('0000-0002-9152-6512', 119), ('0000-0003-3152-116X', 81)]
[('0000-0002-9152-6512', 119), ('0000-0003-3152-116

[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
LR F1:  0.9907032131366733
[('0000-0002-9152-6512', 0.5955056179775281), ('0000-0003-3152-116X', 0.4044943820224719)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
[('0000-0002-9152-6512', 238), ('0000-0003-3152-116X', 162)]
svc F1:  0.9907032131366733
For name:  j_savage
j_savage  pass
For name:  p_matthews
p_matthews  pass
For name:  i_carvalho
i_carvalho  pass
For name:

In [ ]:
%reset

In [ ]:
%who